In [1]:
import pandas as pd
import janitor

from utilities import pandas_to_tex, load_visit_data
import warnings

from constants import filepaths

warnings.filterwarnings("ignore")
# pd.set_option("display.max_columns", 150)

Checking that all paths exist:
{'web_mobile': True, 'web_desktop': True, 'web': True, 'yg_profile': True, 'blacklight': True, 'who': True}


In [2]:
df_visits = load_visit_data()
df_visits.head()

len(df_visits)=6,297,382


,caseid,private_domain,category,visit_time_local,visit_duration,page_views,source
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2,1,mobile_web
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457,10,mobile_web
2,205323077,google.com,Business,2022-06-01 01:15:12,55,1,mobile_web
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225,4,mobile_web
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10,1,mobile_web


In [3]:
# # Get domain-category (using most common cat by visits)
# df_domain_cat = (
#     df_visits.groupby(["private_domain", "category"])
#     .size()
#     .reset_index(name="count")
#     .sort_values(["private_domain", "count"], ascending=(True, False))
#     .drop_duplicates("private_domain", keep="first")
#     .remove_columns("count")
# )
# df_domain_cat

In [4]:
# Get domain traffic from users
df_domain_traffic = (
    df_visits
    .groupby(["private_domain"])
    .size()
    .reset_index(name="traffic")
    .sort_values("traffic", ascending=False, ignore_index=True)
    # Format traffic column (e.g., 823 --> 823, 2750 --> 2.8k, 27503 --> 28k)
    .assign(
        traffic_str=lambda df_: df_["traffic"].apply(
            lambda x: (
                str(x) if x < 1000
                else f"{x / 1000:.1f}k" if x < 10000
                else f"{round(x / 1000)}k"
            )
        )
    ) 
)

dict_domain_traffic = df_domain_traffic.set_index("private_domain")["traffic"].to_dict()

df_domain_traffic

,private_domain,traffic,traffic_str
0,google.com,986722,987k
1,facebook.com,449173,449k
2,yahoo.com,245982,246k
3,bing.com,235769,236k
4,youtube.com,232829,233k
...,...,...,...
64069,vitacost.co,1,1
64070,vitafungus.com,1,1
64071,vistacollege.edu,1,1
64072,visitkokomo.org,1,1


In [5]:
# Get count of trackers per domain
df_blacklight = (
    pd.read_csv(filepaths["blacklight"])
    # Fix filename
    .assign(
        private_domain=lambda df_: df_["filename"].str.replace("_", ".", regex=False)
    )
    .remove_columns("filename")
    .reorder_columns(["private_domain"])
    .set_index("private_domain")
    .add_prefix("bl_")
    .reset_index()
)

df_blacklight

,private_domain,bl_ddg_join_ads,bl_third_party_cookies,bl_canvas_fingerprinting,bl_session_recording,bl_key_logging,bl_fb_pixel,bl_google_analytics
0,costarmanager.com,5,10,0,1,0,0,0
1,teasource.com,11,11,0,0,0,1,1
2,1800tequila.com,8,6,0,0,0,0,0
3,mazon.com,1,0,0,0,0,0,0
4,theancestorhunt.com,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
34073,methowtrails.org,1,0,0,0,0,0,0
34074,mistystoyaussies.com,4,6,0,0,0,0,0
34075,hellostarz.com,0,0,0,0,0,0,0
34076,kbdfans.com,5,7,0,0,0,0,1


In [6]:
# Combine
tracker_cols = [
    "bl_ddg_join_ads",
    "bl_third_party_cookies",
    "bl_fb_pixel",
    "bl_google_analytics",
    "bl_session_recording",
    "bl_key_logging",
    "bl_canvas_fingerprinting",
]

df = (
    df_visits
    .select_columns(["caseid", "private_domain"])
    # ================================================
    # Not longer unique
    # Only get unique visits to domains
#     .drop_duplicates(["caseid", "private_domain"])
    # Get tt unique traffic per domain
    .groupby(["private_domain"])
    .size()
    .reset_index(name="visits")
    .sort_values("visits", ascending=False, ignore_index=True)
    # ================================================
    # Get trackers
    .merge(df_blacklight, how="left", on="private_domain", validate="m:1")
    # Get weighted sum
    .assign(**{k: lambda df_, col=k: df_[col] * df_["visits"] for k in tracker_cols})
    .reorder_columns(["private_domain", "visits", *tracker_cols])
)
df.head(30)

,private_domain,visits,bl_ddg_join_ads,bl_third_party_cookies,bl_fb_pixel,bl_google_analytics,bl_session_recording,bl_key_logging,bl_canvas_fingerprinting
0,google.com,986722,2960166.0,3946888.0,0.0,0.0,0.0,0.0,0.0
1,facebook.com,449173,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,yahoo.com,245982,7625442.0,6887496.0,0.0,0.0,0.0,245982.0,0.0
3,bing.com,235769,0.0,1178845.0,0.0,0.0,0.0,0.0,0.0
4,youtube.com,232829,465658.0,232829.0,0.0,0.0,0.0,0.0,0.0
5,amazon.com,126255,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,twitter.com,111465,111465.0,222930.0,0.0,0.0,0.0,0.0,0.0
7,decipherinc.com,84099,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,live.com,79960,1999000.0,3118440.0,0.0,0.0,0.0,0.0,79960.0
9,reddit.com,61109,61109.0,61109.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_domain_exposure = (
    df.melt(
        id_vars=["private_domain"],
        value_vars=tracker_cols,
        var_name="tracker_type",
        value_name="weighted_tt_exposure",
    )
    .query("weighted_tt_exposure>0")
    .reset_index(drop=True)
)
df_domain_exposure

,private_domain,tracker_type,weighted_tt_exposure
0,google.com,bl_ddg_join_ads,2960166.0
1,yahoo.com,bl_ddg_join_ads,7625442.0
2,youtube.com,bl_ddg_join_ads,465658.0
3,twitter.com,bl_ddg_join_ads,111465.0
4,live.com,bl_ddg_join_ads,1999000.0
...,...,...,...
57857,barnana.com,bl_canvas_fingerprinting,1.0
57858,askwonder.com,bl_canvas_fingerprinting,1.0
57859,asteproallergy.com,bl_canvas_fingerprinting,1.0
57860,alcoveboston.com,bl_canvas_fingerprinting,1.0


In [8]:
# # tt exposure by domain and tracker type
# df_domain_exposure = df_long.groupby(
#     ["tracker_type", "private_domain"], as_index=False
# ).agg(tt_exposure=("weighted_tt_exposure", "sum"))
# df_domain_exposure

In [9]:
_top_n = 50
df_top_domains = (
    df_domain_exposure.sort_values(
        ["tracker_type", "weighted_tt_exposure"], ascending=[True, False], ignore_index=True
    )
    .groupby("tracker_type")
    .head(_top_n)
)
df_top_domains

,private_domain,tracker_type,weighted_tt_exposure
0,live.com,bl_canvas_fingerprinting,79960.0
1,microsoft.com,bl_canvas_fingerprinting,25826.0
2,capitaloneshopping.com,bl_canvas_fingerprinting,23356.0
3,linkedin.com,bl_canvas_fingerprinting,19346.0
4,rakuten.com,bl_canvas_fingerprinting,11539.0
...,...,...,...
39775,bonvoyaged.com,bl_third_party_cookies,79606.0
39776,doceree.com,bl_third_party_cookies,79002.0
39777,verizon.com,bl_third_party_cookies,75399.0
39778,wellsfargo.com,bl_third_party_cookies,74602.0


In [10]:
summary = {}

for tracker in tracker_cols:
    tracker_data = (
        df_top_domains.query("tracker_type == @tracker")
        .merge(df_domain_traffic, how="left", on="private_domain")
        .assign(
            annotate=lambda df_: df_["private_domain"]
            + " ("
            + df_["traffic_str"]
            + ")"
        )["annotate"]
        .tolist()
    )
    summary[tracker] = tracker_data

df_summary = (
    pd.DataFrame.from_dict(summary, orient="index")
    .transpose()
    .assign(ix=lambda df_: range(1, 1 + len(df_)))
    .reorder_columns(["ix"])
)
df_summary

,ix,bl_ddg_join_ads,bl_third_party_cookies,bl_fb_pixel,bl_google_analytics,bl_session_recording,bl_key_logging,bl_canvas_fingerprinting
0,1,yahoo.com (246k),yahoo.com (246k),ebay.com (30k),kohls.com (2.7k),xfinity.com (10k),yahoo.com (246k),live.com (80k)
1,2,google.com (987k),google.com (987k),capitaloneshopping.com (23k),force.com (2.1k),capitalone.com (9.9k),capitaloneshopping.com (23k),microsoft.com (26k)
2,3,live.com (80k),live.com (80k),chase.com (14k),pixiv.net (1.9k),cbssports.com (6.2k),smugmug.com (10k),capitaloneshopping.com (23k)
3,4,aol.com (47k),bing.com (236k),rakuten.com (12k),mheducation.com (1.4k),dell.com (5.5k),weather.com (3.8k),linkedin.com (19k)
4,5,microsoft.com (26k),microsoft.com (26k),hulu.com (11k),tupperware.com (1.4k),att.com (4.9k),activemeasure.com (3.6k),rakuten.com (12k)
5,6,cbssports.com (6.2k),cbssports.com (6.2k),xfinity.com (10k),thriftbooks.com (1.0k),earthlink.net (4.1k),venatusmedia.com (3.5k),hulu.com (11k)
6,7,xfinity.com (10k),xfinity.com (10k),usps.com (9.7k),adp.com (977),venatusmedia.com (3.5k),revenueuniverse.com (3.0k),xfinity.com (10k)
7,8,youtube.com (233k),msn.com (39k),nielseniq.com (9.4k),equitybank.com (888),homedepot.com (3.0k),doceree.com (2.9k),tiktok.com (10.0k)
8,9,ebay.com (30k),ebay.com (30k),netflix.com (7.0k),priceline.com (808),doceree.com (2.9k),spot.im (2.9k),capitalone.com (9.9k)
9,10,imdb.com (7.5k),weather.com (3.8k),wellsfargo.com (6.8k),webtoons.com (705),kohls.com (2.7k),yelp.com (2.3k),washingtonpost.com (8.0k)


In [11]:
pandas_to_tex(
    df_summary,
    texfile="../tables/bl_top_contributors_domain.tex",
    escape=False,
)

In [20]:
pandas_to_tex(
    df_summary.head(10).remove_columns("ix"),
    texfile="../tables/bl_top_contributors_domain_top10.tex",
    escape=False,
)

In [21]:
!cat ../tables/bl_top_contributors_domain_top10.tex

yahoo.com (246k) & yahoo.com (246k) & ebay.com (30k) & kohls.com (2.7k) & xfinity.com (10k) & yahoo.com (246k) & live.com (80k) \\
google.com (987k) & google.com (987k) & capitaloneshopping.com (23k) & force.com (2.1k) & capitalone.com (9.9k) & capitaloneshopping.com (23k) & microsoft.com (26k) \\
live.com (80k) & live.com (80k) & chase.com (14k) & pixiv.net (1.9k) & cbssports.com (6.2k) & smugmug.com (10k) & capitaloneshopping.com (23k) \\
aol.com (47k) & bing.com (236k) & rakuten.com (12k) & mheducation.com (1.4k) & dell.com (5.5k) & weather.com (3.8k) & linkedin.com (19k) \\
microsoft.com (26k) & microsoft.com (26k) & hulu.com (11k) & tupperware.com (1.4k) & att.com (4.9k) & activemeasure.com (3.6k) & rakuten.com (12k) \\
cbssports.com (6.2k) & cbssports.com (6.2k) & xfinity.com (10k) & thriftbooks.com (1.0k) & earthlink.net (4.1k) & venatusmedia.com (3.5k) & hulu.com (11k) \\
xfinity.com (10k) & xfinity.com (10k) & usps.com (9.7k) & adp.com (977) & venatusmedia.com (3.5k) & revenue

In [13]:
!cat ../tables/bl_top_contributors_domain.tex

1 & yahoo.com (246k) & yahoo.com (246k) & ebay.com (30k) & kohls.com (2.7k) & xfinity.com (10k) & yahoo.com (246k) & live.com (80k) \\
2 & google.com (987k) & google.com (987k) & capitaloneshopping.com (23k) & force.com (2.1k) & capitalone.com (9.9k) & capitaloneshopping.com (23k) & microsoft.com (26k) \\
3 & live.com (80k) & live.com (80k) & chase.com (14k) & pixiv.net (1.9k) & cbssports.com (6.2k) & smugmug.com (10k) & capitaloneshopping.com (23k) \\
4 & aol.com (47k) & bing.com (236k) & rakuten.com (12k) & mheducation.com (1.4k) & dell.com (5.5k) & weather.com (3.8k) & linkedin.com (19k) \\
5 & microsoft.com (26k) & microsoft.com (26k) & hulu.com (11k) & tupperware.com (1.4k) & att.com (4.9k) & activemeasure.com (3.6k) & rakuten.com (12k) \\
6 & cbssports.com (6.2k) & cbssports.com (6.2k) & xfinity.com (10k) & thriftbooks.com (1.0k) & earthlink.net (4.1k) & venatusmedia.com (3.5k) & hulu.com (11k) \\
7 & xfinity.com (10k) & xfinity.com (10k) & usps.com (9.7k) & adp.com (977) & venat